In [23]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from pinecone import PineconeApiException
from pinecone import Index

In [14]:
os.environ["PINECONE_API_KEY"] = "8cab3ffc-f443-427e-8458-3bb4c96ceaa4"

In [15]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '470a54b0235f7c1ad8b8f9d40ed40eac', 'Date': 'Sat, 24 Aug 2024 10:59:22 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [ ]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'name': 'Prof. Vijaysekhar Chellaboina',
  'designation': 'Professor and Dean, School of Computer Science',
  'profile_image': '/assets/images/icon-sidebar-form.svg',
  'department': 'School of Computer Science',
  'profile_summary': 'Vijay earned his B. Tech in Mechanical Engineering from IIT Madras and his M.S. in Mechanical Engineering from Florida Institute of Technology. He holds a Ph.D. in Aerospace Engineering from the Georgia Institute of Technology. With a solid foundation in mechanical and aerospace engineering and research expertise in applied mathematics, his teaching interests encompass mathematics, mechanical, aerospace, and electrical engineering, as well as computer science.His research focuses on stability theory and feedback control applications to complex systems, including biomedical and financial systems. Vijay has contributed over a hundred journal papers and a similar number of peer-reviewed conference articles, and has authored five books published by Springer

In [ ]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review.get('profile_summary', ""),  # Using the profile summary as input for embedding
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review.get("name", "Unknown"),  # Use the professor's name as the unique identifier
        "metadata": {
            "name": review.get("name", "Unknown"),  # Professor's name
            "designation": review.get("designation", "Unknown"),  # Professor's designation
            "profile_image": review.get("profile_image", ""),  # URL to professor's profile image
            "department": review.get("department", "School of Computer Science"),  # Department name
            "profile_summary": review.get("profile_summary", "N/A"),  # Summary of professor's profile
            "work_experience": review.get("work_experience", "N/A"),  # Professor's work experience
            "research_interests": review.get("research_interests", "N/A"),  # Research interests
            "teaching_philosophy": review.get("teaching_philosophy", "N/A"),  # Teaching philosophy
            "courses_taught": review.get("courses_taught", "N/A"),  # Courses taught by the professor
            "awards_and_grants": review.get("awards_and_grants", "N/A"),  # Awards and grants received
            "scholarly_activities": review.get("scholarly_activities", "N/A"),  # Scholarly activities
            "contact": review.get("contact", "N/A")  # Contact information
        }
    })

# Now, processed_data contains all the processed review data


In [21]:
index = pc.Index('rag')

# Define the batch size (adjust as needed)
batch_size = 100  # This can be adjusted to fit within the size limit

# Function to chunk data into batches
def chunk_data(data, size):
    for i in range(0, len(data), size):
        yield data[i:i + size]

# Upload data in batches
for batch in chunk_data(processed_data, batch_size):
    try:
        index.upsert(
            vectors=batch,
            namespace="ns1"
        )
    except PineconeApiException as e:
        print(f"Error during upsert: {e}")


Error during upsert: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 24 Aug 2024 11:10:17 GMT', 'Content-Type': 'application/json', 'Content-Length': '115', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '444', 'x-pinecone-request-id': '183091092155953620', 'x-envoy-upstream-service-time': '7', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Metadata size is 41200 bytes, which exceeds the limit of 40960 bytes per vector","details":[]}

